In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter


# Open all the Excel Files

In [2]:
SchedulePath = 'Input/MD_10/RSC_Schedule.xlsx'
RPVPath = 'Input/MD_10/RSC_RPV.xlsx'
SBVPath = 'Input/MD_10/RSC_SBV.xlsx'
IDRPath = 'Input/MD_10/RSC_IDR.xlsx'

ScheduleSheet = pd.ExcelFile(SchedulePath)
RPVSheet = pd.ExcelFile(RPVPath)
SBVSheet = pd.ExcelFile(SBVPath)
IDRSheet = pd.ExcelFile(IDRPath)

# List out all the Pages in each Excel sheet

In [3]:
schedulePages = ScheduleSheet.sheet_names
RPVPages = RPVSheet.sheet_names
SBVPages = SBVSheet.sheet_names
IDRPages = IDRSheet.sheet_names

print("Schedule Pages\n-----------------")
for page in schedulePages:
    print(page)
print("\n")

print("RPV Pages\n-----------------")
for page in RPVPages:
    print(page)
print("\n")

print("SBV Pages\n-----------------")
for page in SBVPages:
    print(page)
print("\n")

print("IDR Pages\n-----------------")
for page in IDRPages:
    print(page)
print("\n")


Schedule Pages
-----------------
Match Day Schedules
Premier
Master
Elite
Veteran
Rival
Challenger
Prospect
Contender
Amateur
MasterSchedule
Working sheet
DataValidation
F&T


RPV Pages
-----------------
Glossary
All-Star Candidates
MVP Candidates
Sorted RPV
MVP Eligible Teams
Aggregate RPV
IDR Import
Variables
Raw_Stats
Contracts
Members
Teams Info


SBV Pages
-----------------
MVP Candidates
All-Star Candidates
Glossary
Sorted Team Stats
Sorted Player Stats
Sheet20
Team Stats
Player Stats
Multiuse Variables
Imported Game Data
Members
Variables
IDR Import
MVP Eligible Teams
Teams Info


IDR Pages
-----------------
Glossary
IDR View
Aggregate IDR
Raw Stats Imports
SBV Import
Constants




# Extract the Schedule Page

In [4]:
MD_Schedules = ScheduleSheet.parse(schedulePages[0])

#
# Bundle all this into a Class for a Tier?
#

# Define the Function for Extracting the Tables
def GetTable (sheet, coords):
    return sheet.parse(sheet.sheet_names[0], skiprows=coords["skiprows"], usecols=coords["usecols"], nrows=coords["nrows"])

PremiereCoords = { "skiprows": 5, "usecols": "B:I", "nrows": 49-6 }
MasterCoords = { "skiprows": 56, "usecols": "B:K", "nrows": 102-57 }
EliteCoords = { "skiprows": 109, "usecols": "B:Q", "nrows": 158-110 }
VeteranCoords = { "skiprows": 167, "usecols": "B:V", "nrows": 218-168 } # Ignoring Divisions
RivalCoords = { "skiprows": 227, "usecols": "B:V", "nrows": 278-228 }
ChallengerCoords = { "skiprows": 285, "usecols": "B:Q", "nrows": 334-286 }
ProspectCoords = { "skiprows": 341, "usecols": "B:K", "nrows": 387-342 }
ContenderCoords = { "skiprows": 394, "usecols": "B:H", "nrows": 438-395 }
AmateurCoords = { "skiprows": 445, "usecols": "B:K", "nrows": 464-446 }

PremiereSchedule = GetTable(ScheduleSheet, PremiereCoords)
MasterSchedule = GetTable(ScheduleSheet, MasterCoords)
EliteSchedule = GetTable(ScheduleSheet, EliteCoords)
VeteranSchedule = GetTable(ScheduleSheet, VeteranCoords)
RivalSchedule = GetTable(ScheduleSheet, RivalCoords)
ChallengerSchedule = GetTable(ScheduleSheet, ChallengerCoords)
ProspectSchedule = GetTable(ScheduleSheet, ProspectCoords)
ContenderSchedule = GetTable(ScheduleSheet, ContenderCoords)
AmateurSchedule = GetTable(ScheduleSheet, AmateurCoords)

PremiereTeamNum = (len(PremiereSchedule.columns) - 2) * 2
MasterTeamNum = (len(MasterSchedule.columns) - 2) * 2
EliteTeamNum = (len(EliteSchedule.columns) - 2) * 2
VeteranTeamNum = (len(VeteranSchedule.columns) - 2 - 3) * 2
RivalTeamNum = (len(RivalSchedule.columns) - 2 - 3) * 2
ChallengerTeamNum = (len(ChallengerSchedule.columns) - 2) * 2
ProspectTeamNum = (len(ProspectSchedule.columns) - 2) * 2
ContenderTeamNum = (len(ContenderSchedule.columns) - 2) * 2
AmateurTeamNum = (len(AmateurSchedule.columns) - 2)

print("Premiere Teams: ", PremiereTeamNum)
print("Master Teams: ", MasterTeamNum)
print("Elite Teams: ", EliteTeamNum)
print("Veteran Teams: ", VeteranTeamNum)
print("Rival Teams: ", RivalTeamNum)
print("Challenger Teams: ", ChallengerTeamNum)
print("Prospect Teams: ", ProspectTeamNum)
print("Contender Teams: ", ContenderTeamNum)
print("Amateur Teams: ", AmateurTeamNum)

#TODO Split these into the Solar and Lunar Franchises

Premiere Teams:  12
Master Teams:  16
Elite Teams:  28
Veteran Teams:  32
Rival Teams:  32
Challenger Teams:  28
Prospect Teams:  16
Contender Teams:  10
Amateur Teams:  8


# Extracting the Scrim Dates and infos

In [5]:
# Extract the Scrim Schedules
# Sheets go from Columns A:L A-L = 12
# First 3 are day/date Related
AtoL = 12 # 12 Columns
TeamsPerRow = AtoL - 3

PremiereScrimSheetName = schedulePages[1]
MasterScrimSheetName = schedulePages[2]
EliteScrimSheetName = schedulePages[3]
VeteranScrimSheetName = schedulePages[4]
RivalScrimSheetName = schedulePages[5]
ChallengerScrimSheetName = schedulePages[6]
ProspectScrimSheetName = schedulePages[7]
ContenderScrimSheetName = schedulePages[8]
AmateurScrimSheetName = schedulePages[9]

print(PremiereScrimSheetName)
print(MasterScrimSheetName)
print(EliteScrimSheetName)
print(VeteranScrimSheetName)
print(RivalScrimSheetName)
print(ChallengerScrimSheetName)
print(ProspectScrimSheetName)
print(ContenderScrimSheetName)
print(AmateurScrimSheetName)

PremiereScrimRows = int(PremiereTeamNum / TeamsPerRow) + 1
MasterScrimRows = int(MasterTeamNum / TeamsPerRow)+ 1
EliteScrimRows = int(EliteTeamNum / TeamsPerRow)+ 1
VeteranScrimRows = int(VeteranTeamNum / TeamsPerRow)+ 1
RivalScrimRows = int(RivalTeamNum / TeamsPerRow)+ 1
ChallengerScrimRows = int(ChallengerTeamNum / TeamsPerRow)+ 1
ProspectScrimRows = int(ProspectTeamNum / TeamsPerRow)+ 1
ContenderScrimRows = int(ContenderTeamNum / TeamsPerRow)+ 1
AmateurScrimRows = int(AmateurTeamNum / TeamsPerRow)+ 1

# Start Extraction at Row 14
BufferPerRow = 2
GamesPerSeason = 16
RowHeight = GamesPerSeason + 1 + BufferPerRow

def GetTable (sheet, sheetName, coords):
    return sheet.parse(sheetName, skiprows=coords["skiprows"], usecols=coords["usecols"], nrows=coords["nrows"])

PremiereCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 49-14 }
MasterCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 49-14 }
EliteCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 87-14 }
VeteranCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 87-14 } # Ignoring Divisions
RivalCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 87-14 }
ChallengerCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 87-14 }
ProspectCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 49-14 }
ContenderCoords = { "skiprows": 13, "usecols": "B:L", "nrows": 49-14 }
AmateurCoords = { "skiprows": 13, "usecols": "B:K", "nrows": 30-14 }

PremiereScrimSheet = GetTable(ScheduleSheet,PremiereScrimSheetName, PremiereCoords)
MasterScrimSheet = GetTable(ScheduleSheet,MasterScrimSheetName, MasterCoords)
EliteScrimSheet = GetTable(ScheduleSheet,EliteScrimSheetName, EliteCoords)
VeteranScrimSheet = GetTable(ScheduleSheet,VeteranScrimSheetName, VeteranCoords)
RivalScrimSheet = GetTable(ScheduleSheet,RivalScrimSheetName, RivalCoords)
ChallengerScrimSheet = GetTable(ScheduleSheet,ChallengerScrimSheetName, ChallengerCoords)
ProspectScrimSheet = GetTable(ScheduleSheet,ProspectScrimSheetName, ProspectCoords)
ContenderScrimSheet = GetTable(ScheduleSheet,ContenderScrimSheetName, ContenderCoords)
AmateurScrimSheet = GetTable(ScheduleSheet,AmateurScrimSheetName, AmateurCoords)

#def GetScrimTable (excelSheet, sheetName, tierTeamsNum, teamsPerRow = 9, bufferPerRow = 2, gamesPerSeason = 16, startRow = 14):
#    RowHeight = gamesPerSeason + 1 + bufferPerRow
#    ScheduleRows = int(tierTeamsNum / teamsPerRow) + 1
#    TotalRows = ScheduleRows * RowHeight
#    return excelSheet.parse(sheetName, skiprows=startRow-1, nrows=TotalRows, usecols="A:L")


Premier
Master
Elite
Veteran
Rival
Challenger
Prospect
Contender
Amateur


# Linearize the Match Day Schedules

In [6]:
def LinearizeMatchDaySchedule (sheet, preseasonDays, matchDays,  playoffDays, conferencesPerTier=2, spacing=4, hasLunarConference=True):
    TotalDays = preseasonDays + matchDays
    
    OfficialSchedule = sheet.iloc[:TotalDays, [0, -1]]
    OfficialSchedule.columns = ["Match Day", "Date"]
    
    SolarConference = sheet.iloc[:TotalDays, 1:-1]
    SolarConference = SolarConference.dropna(axis=1, how='all')
    
    # Fill in Playoff Days
    for i in range(TotalDays - playoffDays, TotalDays):
        DayValue = SolarConference.iloc[i, 0]
        SolarConference.iloc[i, :] = DayValue
        
    OfficialSchedule = pd.concat([OfficialSchedule, SolarConference], axis=1)

    if hasLunarConference:
        LunarConferenceTeamNames = sheet.iloc[TotalDays + spacing, 1:-1].values
        LunarConference = sheet.iloc[TotalDays + spacing+1:TotalDays + spacing + TotalDays + 1, 1:-1]
        LunarConference.columns = LunarConferenceTeamNames
        LunarConference.reset_index(drop=True, inplace=True)
        LunarConference = LunarConference.dropna(axis=1, how='all')
        
        # Fill in Playoff Days
        for i in range(TotalDays - playoffDays, TotalDays):
            DayValue = LunarConference.iloc[i, 0]
            LunarConference.iloc[i, :] = DayValue
            
        OfficialSchedule = pd.concat([OfficialSchedule, LunarConference], axis=1)
        
        OfficialSchedule = OfficialSchedule.dropna(axis=1, how='all')

    return OfficialSchedule

MatchPremiereSchedule =    LinearizeMatchDaySchedule (PremiereSchedule, 3, 16, 2)
MatchMasterSchedule =      LinearizeMatchDaySchedule (MasterSchedule, 3, 17, 2)
MatchEliteSchedule =       LinearizeMatchDaySchedule (EliteSchedule, 3, 18, 3, spacing=5)
MatchVeteranSchedule =     LinearizeMatchDaySchedule (VeteranSchedule, 3, 19, 3, spacing=5)
MatchRivalSchedule =       LinearizeMatchDaySchedule (RivalSchedule, 3, 19, 3, spacing=5)
MatchChallengerSchedule =  LinearizeMatchDaySchedule (ChallengerSchedule, 3, 18, 3, spacing=5)
MatchProspectSchedule =    LinearizeMatchDaySchedule (ProspectSchedule, 3, 17, 2)
MatchContenderSchedule =   LinearizeMatchDaySchedule (ContenderSchedule, 3, 16, 3)
MatchAmateurSchedule =     LinearizeMatchDaySchedule (AmateurSchedule, 3, 15, 1, hasLunarConference=False)

MatchSchedules = { "Premier": MatchPremiereSchedule, "Master": MatchMasterSchedule, "Elite": MatchEliteSchedule, "Veteran": MatchVeteranSchedule, "Rival": MatchRivalSchedule, "Challenger": MatchChallengerSchedule, "Prospect": MatchProspectSchedule, "Contender": MatchContenderSchedule, "Amateur": MatchAmateurSchedule }

# Linearize the Scrim Schedule

In [7]:
def LinearizeScrimDaySchedule (sheet, numOfRows, matchDays=16, spacing=2):
    OfficialSheet = sheet.iloc[:matchDays, [0, 2]]
    FirstRow = sheet.iloc[:matchDays, 3:]
    OfficialSheet = pd.concat([OfficialSheet, FirstRow], axis=1)
    
    Start = matchDays + spacing
    
    for i in range(1, numOfRows):
        RowNames = sheet.iloc[Start][2:].values
        Row = sheet.iloc[(Start + 1):(Start + 1) + matchDays, 2:]
        Row.columns = RowNames
        Row.reset_index(drop=True, inplace=True)
        Row.dropna(axis=1, how='all')
        Row.dropna(axis=0, how='all')
        
        Start += (matchDays + spacing + 1) 
        
        OfficialSheet = pd.concat([OfficialSheet, Row], axis=1)
    
    OfficialSheet = OfficialSheet.loc[:, ~OfficialSheet.columns.isna()]
    ScrimNumbers = len(OfficialSheet["Game"].values)
    OfficialSheet["Game"] =  ["Match Day " + str(i) for i in range(1, ScrimNumbers + 1)]
    OfficialSheet.rename(columns={"Game": "Match Day"}, inplace=True)
    
    return OfficialSheet

ScrimPremiereSchedule =    LinearizeScrimDaySchedule(PremiereScrimSheet, PremiereScrimRows)
ScrimMasterSchedule =      LinearizeScrimDaySchedule(MasterScrimSheet, MasterScrimRows, matchDays=16, spacing=2)
ScrimEliteSchedule =       LinearizeScrimDaySchedule(EliteScrimSheet, EliteScrimRows)
ScrimVeteranSchedule =     LinearizeScrimDaySchedule(VeteranScrimSheet, VeteranScrimRows)
ScrimRivalSchedule =       LinearizeScrimDaySchedule(RivalScrimSheet, RivalScrimRows)
ScrimChallengerSchedule =  LinearizeScrimDaySchedule(ChallengerScrimSheet, ChallengerScrimRows)
ScrimProspectSchedule =    LinearizeScrimDaySchedule(ProspectScrimSheet, ProspectScrimRows)
ScrimContenderSchedule =   LinearizeScrimDaySchedule(ContenderScrimSheet, ContenderScrimRows)
ScrimAmateurSchedule =     LinearizeScrimDaySchedule(AmateurScrimSheet, AmateurScrimRows)

ScrimSchedules = { "Premier": ScrimPremiereSchedule, "Master": ScrimMasterSchedule, "Elite": ScrimEliteSchedule, "Veteran": ScrimVeteranSchedule, "Rival": ScrimRivalSchedule, "Challenger": ScrimChallengerSchedule, "Prospect": ScrimProspectSchedule, "Contender": ScrimContenderSchedule, "Amateur": ScrimAmateurSchedule }


# Combine the Schedules

In [8]:
#Tiers = ["Premier", "Master", "Elite", "Veteran", "Rival", "Challenger", "Prospect", "Amateur"]
#
#display(ScrimSchedules[Tiers[0]])
#
##for tier in Tiers:
##    ScrimSchedules[tier]["Game"] = ["Scrim Day " + str(game) for game in ScrimSchedules[tier]["Game"]]
#
#display(ScrimSchedules[Tiers[0]])
#print(ScrimSchedules[Tiers[0]]["Game"].values)
#ScrimNumbers = len(ScrimSchedules[Tiers[0]]["Game"].values)
#
#print(ScrimNumbers)
#
#Titles = ["Scrim Day " + str(i) for i in range(1, ScrimNumbers + 1)]
#
#NewTable = ScrimSchedules[Tiers[0]]
#NewTable.drop("Game", axis=1, inplace=True)
#NewTable = NewTable.T
#NewTable.columns = Titles
#
#display(NewTable)
#
#display(ScrimSchedules[Tiers[0]])
#display(ScrimSchedules[Tiers[0]].T)
#
#print([col for col in MatchSchedules[Tiers[0]].T.columns])
#print([col for col in ScrimSchedules[Tiers[0]].columns])
#
##PreseasonDays = [col for col in ScrimSchedules[Tiers[0]].columns if "Preseason" in col]
#
##print(PreseasonDays)


# Extracting Raw Data Tables (RPV, IDR and SBV)

In [9]:
Tiers = ["Premier", "Master", "Elite", "Veteran", "Rival", "Challenger", "Prospect", "Amateur"]

SortedRPVSheet = RPVSheet.parse("Sorted RPV", skiprows=4, usecols="A:AD", nrows=848)
SortedSBVSheet = SBVSheet.parse("Sorted Player Stats", skiprows=0, usecols="D:AQ", nrows=848)   
SortedIDRSheet = IDRSheet.parse("IDR View", skiprows=4, usecols="A:AQ", nrows=848)   

def GetTierTables (sheet):

    TierTables = []

    for tier in Tiers:
        tierTable = sheet[sheet["Tier"] == tier]
        tierTable.reset_index(drop=True, inplace=True)
        tierTable.columns = tierTable.columns.str.strip()
        TierTables.append(tierTable)
        
    return TierTables

def GetMergedTierTables (RPVSheet, SBVSheet, IDRSheet):
    RPVTables = GetTierTables(RPVSheet)
    SBVTables = GetTierTables(SBVSheet)
    IDRTables = GetTierTables(IDRSheet)
    
    MergedTables = []
    
    for i in range(len(Tiers)):
        mergedTable = (
            RPVTables[i]
            .merge(SBVTables[i], on=["Name"])
            .merge(IDRTables[i], on=["Name"])
            )
        
        mergedTable = mergedTable.rename(columns={col: col.replace("_x", "") for col in mergedTable.columns if "_x" in col})
        mergedTable = mergedTable.rename(columns={col: col.replace("(s)", "_y") for col in mergedTable.columns if "(s)" in col})

        # Remove columns containing "_y" or "_z"
        remove_cols = [col for col in mergedTable.columns if "_y" in col or "_z" in col]
        mergedTable = mergedTable.drop(columns=remove_cols)
        
        mergedTable = mergedTable.loc[:, ~mergedTable.columns.duplicated()]
        
        MergedTables.append(mergedTable)
    
   
    
    MergedTables
    
    return MergedTables

TierTables = GetMergedTierTables(SortedRPVSheet, SortedSBVSheet, SortedIDRSheet)

StatsTable = pd.concat(TierTables, keys=Tiers)

# Save all the Results to an Excel File

In [ ]:
def get_excel_range(start_row, start_col, df):
    rows, cols = df.shape
    end_row = start_row + rows  # Bottom-right row
    end_col = start_col + cols - 1  # Bottom-right column (adjust for zero indexing)
    
    
    start_cell = f"{get_column_letter(start_col)}{start_row}"
    end_cell = f"{get_column_letter(end_col)}{end_row}" # Index = False, Headers = True, adjust based off that
    
    print(f"{start_cell}:{end_cell}")
    
    return f"{start_cell}:{end_cell}"

def PasteDataFrame(dataFrame, workSheet, startRow = 1, startCol = 1):
    for j, col_name in enumerate(dataFrame.columns, start=startCol):  # Write column headers
        workSheet.cell(row=startRow, column=j).value = col_name
    
    for row_num, row_data in enumerate(dataFrame.itertuples(index=False), start=startRow + 1):  # Write data
        for col_num, value in enumerate(row_data, start=startCol):
            workSheet.cell(row=row_num, column=col_num).value = value

def AddDataFrameAsTable (dataFrame, workSheet, tableName, startRow = 1, startCol = 1):
    PasteDataFrame(dataFrame, workSheet, startRow, startCol)
    
    workSheet.add_table(Table(displayName=tableName, ref=get_excel_range(startRow, startCol, dataFrame)))
    
#WorkBook = load_workbook("RSC_Info.xlsx")
WorkBook = Workbook()
WorkSheet = WorkBook.active
WorkSheet.title = "Schedules"

rowIndex = 1

for i in range(len(Tiers)):
    
    WorkSheet.merge_cells(start_row=rowIndex, start_column=1, end_row=rowIndex + 1, end_column=MatchSchedules[Tiers[i]].columns.size)
    WorkSheet.cell(row=rowIndex, column=1).value = Tiers[i] + " Match Schedule"
    WorkSheet.cell(row=rowIndex, column=1).alignment = Alignment(horizontal='center', vertical='center')

    rowIndex += 2
    
    # Paste Match Schedule DataFrame
    AddDataFrameAsTable(MatchSchedules[Tiers[i]], WorkSheet, Tiers[i] + "MatchSchedule", startRow=rowIndex)
    
    rowIndex += MatchSchedules[Tiers[i]].shape[0] + 1
    
    WorkSheet.merge_cells(start_row=rowIndex, start_column=1, end_row=rowIndex, end_column=MatchSchedules[Tiers[i]].columns.size)
    WorkSheet.cell(row=rowIndex, column=1).fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
    
    rowIndex += 1
    
    # Scrim
    WorkSheet.merge_cells(start_row=rowIndex, start_column=1, end_row=rowIndex + 1, end_column=ScrimSchedules[Tiers[i]].columns.size)
    WorkSheet.cell(row=rowIndex, column=1).value = Tiers[i] + " Scrim Schedule"
    WorkSheet.cell(row=rowIndex, column=1).alignment = Alignment(horizontal='center', vertical='center')
    
    rowIndex += 2
    
    AddDataFrameAsTable(ScrimSchedules[Tiers[i]], WorkSheet, Tiers[i] + "ScrimSchedule", startRow=rowIndex)
    
    rowIndex += ScrimSchedules[Tiers[i]].shape[0] + 1
    
    WorkSheet.merge_cells(start_row=rowIndex, start_column=1, end_row=rowIndex, end_column=ScrimSchedules[Tiers[i]].columns.size)
    WorkSheet.cell(row=rowIndex, column=1).fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
    
    rowIndex += 1

#PasteDataFrame(StatsTable, WorkSheet)
print(f"StatsTable shape: {StatsTable.shape}")


WorkBook.create_sheet("Stats")

WorkSheet = WorkBook["Stats"]

AddDataFrameAsTable(StatsTable, WorkSheet, "Stats")

WorkBook.save("RSC_Info.xlsx")

#for i in range(60, len(StatsTable.columns)):
#    
#    NewWorkBook = WorkBook
#    
#    NewWorkBook.create_sheet(f"Stats_{i}")
#
#    WorkSheet = NewWorkBook[f"Stats_{i}"]
#    
#    NewTable = StatsTable.iloc[:, 0:i]
#
#    display(NewTable)
#
#    AddDataFrameAsTable(NewTable, WorkSheet, f"Stats_{i}")
#
#    NewWorkBook.save(f"RSC_Info_{i}.xlsx")
#    
#    NewWorkBook.remove_sheet(WorkSheet)


A3:N22
A26:M42
A46:R66
A70:Q86
A90:AD111
A115:AC131
A135:AH157
A161:AG177
A181:AH203
A207:AG223
A227:AD248
A252:AC268
A272:R292
A296:Q312
A316:J334
A338:I354
StatsTable shape: (791, 91)


Tier     RSC ID           Name                   Team  \
Premier 0   Premier  RSC005157          ScuLa             Excavators   
        1   Premier  RSC007982        Cadomac             Hurricanes   
        2   Premier  RSC004216           GrEh            Long Beards   
        3   Premier  RSC004459        phoenix            Long Beards   
        4   Premier  RSC007460          mogii            Gummy Bears   
...             ...        ...            ...                    ...   
Amateur 31  Amateur  RSC006022   N0MrThomasMn  Aloof Goofs\nSurprise   
        32  Amateur  RSC007393    Ubfghki9442           Garlic Knots   
        33  Amateur  RSC007678      SavedHawk                  Kiwis   
        34  Amateur  RSC000968  Tsg_raven5336                 Hyenas   
        35  Amateur  RSC007587   ShortyoShire  Aloof Goofs\nSurprise   

           Conference        SBV        IDR        RPV  GP   W  ...  \
Premier 0         NaN  82.427287  91.800467  87.113877  34  25  ...   
        1         NaN  72.945213  94.910885  83.928049  31  19  ...   
        2         NaN  66.698918  94.988624  80.843771  36  24  ...   
        3         NaN  68.476948  90.874852  79.675900  36  24  ...   
        4         NaN  75.613399  77.750531  76.681965  28  15  ...   
...               ...        ...        ...        ...  ..  ..  ...   
Amateur 31        NaN  25.506401  12.596647  19.051524   4   0  ...   
        32        NaN   2.773334  28.030267  15.401800   6   3  ...   
        33        NaN   7.391318   7.616544   7.503931  24   4  ...   
        34        NaN  -7.766953   5.115930  -1.325512   6   3  ...   
        35        NaN -15.162277 -29.180718 -22.171497   6   1  ...   

                  NP/G       G/G       A/G      Sv/G       S/G  tm_G  opp_G  \
Premier 0   188.588235  1.147059  0.882353  1.558824  3.117647   104     66   
        1   195.451613  1.000000  0.935484  1.645161  3.774194    87     64   
        2   147.138889  1.194444  0.972222  0.972222  3.611111   117     77   
        3   151.972222  1.111111  0.888889  1.805556  3.638889   117     77   
        4   169.428571  1.142857  0.642857  1.714286  3.392857    70     71   
...                ...       ...       ...       ...       ...   ...    ...   
Amateur 31  108.000000  0.750000  0.250000  0.500000  1.250000     5     18   
        32   73.166667  0.500000  0.333333  0.166667  1.000000    14     12   
        33   90.416667  0.208333  0.291667  0.750000  0.958333    32     93   
        34  111.666667  0.000000  0.000000  0.833333  0.666667    10      9   
        35   73.833333  0.000000  0.000000  0.833333  0.666667     8     29   

              tm_G/A   opp_G/A    tm_G/G  
Premier 0   3.107930  1.972340  3.058824  
        1   2.549196  1.875271  2.806452  
        2   3.011715  1.982069  3.250000  
        3   3.011715  1.982069  3.250000  
        4   2.559309  2.595870  2.500000  
...              ...       ...       ...  
Amateur 31  1.337191  4.813888  1.250000  
        32  2.266187  1.942446  2.333333  
        33  1.307871  3.800999  1.333333  
        34  1.708633  1.537770  1.666667  
        35  1.348182  4.887159  1.333333  

[791 rows x 60 columns]

A1:BH792


C:\Users\MrDNA\AppData\Local\Temp\ipykernel_34432\3525909124.py:87: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  NewWorkBook.remove_sheet(WorkSheet)


Tier     RSC ID           Name                   Team  \
Premier 0   Premier  RSC005157          ScuLa             Excavators   
        1   Premier  RSC007982        Cadomac             Hurricanes   
        2   Premier  RSC004216           GrEh            Long Beards   
        3   Premier  RSC004459        phoenix            Long Beards   
        4   Premier  RSC007460          mogii            Gummy Bears   
...             ...        ...            ...                    ...   
Amateur 31  Amateur  RSC006022   N0MrThomasMn  Aloof Goofs\nSurprise   
        32  Amateur  RSC007393    Ubfghki9442           Garlic Knots   
        33  Amateur  RSC007678      SavedHawk                  Kiwis   
        34  Amateur  RSC000968  Tsg_raven5336                 Hyenas   
        35  Amateur  RSC007587   ShortyoShire  Aloof Goofs\nSurprise   

           Conference        SBV        IDR        RPV  GP   W  ...       G/G  \
Premier 0         NaN  82.427287  91.800467  87.113877  34  25  ...  1.147059   
        1         NaN  72.945213  94.910885  83.928049  31  19  ...  1.000000   
        2         NaN  66.698918  94.988624  80.843771  36  24  ...  1.194444   
        3         NaN  68.476948  90.874852  79.675900  36  24  ...  1.111111   
        4         NaN  75.613399  77.750531  76.681965  28  15  ...  1.142857   
...               ...        ...        ...        ...  ..  ..  ...       ...   
Amateur 31        NaN  25.506401  12.596647  19.051524   4   0  ...  0.750000   
        32        NaN   2.773334  28.030267  15.401800   6   3  ...  0.500000   
        33        NaN   7.391318   7.616544   7.503931  24   4  ...  0.208333   
        34        NaN  -7.766953   5.115930  -1.325512   6   3  ...  0.000000   
        35        NaN -15.162277 -29.180718 -22.171497   6   1  ...  0.000000   

                 A/G      Sv/G       S/G  tm_G  opp_G    tm_G/A   opp_G/A  \
Premier 0   0.882353  1.558824  3.117647   104     66  3.107930  1.972340   
        1   0.935484  1.645161  3.774194    87     64  2.549196  1.875271   
        2   0.972222  0.972222  3.611111   117     77  3.011715  1.982069   
        3   0.888889  1.805556  3.638889   117     77  3.011715  1.982069   
        4   0.642857  1.714286  3.392857    70     71  2.559309  2.595870   
...              ...       ...       ...   ...    ...       ...       ...   
Amateur 31  0.250000  0.500000  1.250000     5     18  1.337191  4.813888   
        32  0.333333  0.166667  1.000000    14     12  2.266187  1.942446   
        33  0.291667  0.750000  0.958333    32     93  1.307871  3.800999   
        34  0.000000  0.833333  0.666667    10      9  1.708633  1.537770   
        35  0.000000  0.833333  0.666667     8     29  1.348182  4.887159   

              tm_G/G   opp_G/G  
Premier 0   3.058824  1.941176  
        1   2.806452  2.064516  
        2   3.250000  2.138889  
        3   3.250000  2.138889  
        4   2.500000  2.535714  
...              ...       ...  
Amateur 31  1.250000  4.500000  
        32  2.333333  2.000000  
        33  1.333333  3.875000  
        34  1.666667  1.500000  
        35  1.333333  4.833333  

[791 rows x 61 columns]

...

A1:CK792


C:\Users\MrDNA\AppData\Local\Temp\ipykernel_34432\3525909124.py:87: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  NewWorkBook.remove_sheet(WorkSheet)


Tier     RSC ID           Name                   Team  \
Premier 0   Premier  RSC005157          ScuLa             Excavators   
        1   Premier  RSC007982        Cadomac             Hurricanes   
        2   Premier  RSC004216           GrEh            Long Beards   
        3   Premier  RSC004459        phoenix            Long Beards   
        4   Premier  RSC007460          mogii            Gummy Bears   
...             ...        ...            ...                    ...   
Amateur 31  Amateur  RSC006022   N0MrThomasMn  Aloof Goofs\nSurprise   
        32  Amateur  RSC007393    Ubfghki9442           Garlic Knots   
        33  Amateur  RSC007678      SavedHawk                  Kiwis   
        34  Amateur  RSC000968  Tsg_raven5336                 Hyenas   
        35  Amateur  RSC007587   ShortyoShire  Aloof Goofs\nSurprise   

           Conference        SBV        IDR        RPV  GP   W  ...    OShAPG  \
Premier 0         NaN  82.427287  91.800467  87.113877  34  25  ...  2.837500   
        1         NaN  72.945213  94.910885  83.928049  31  19  ...  2.801680   
        2         NaN  66.698918  94.988624  80.843771  36  24  ...  2.813488   
        3         NaN  68.476948  90.874852  79.675900  36  24  ...  2.813488   
        4         NaN  75.613399  77.750531  76.681965  28  15  ...  2.764711   
...               ...        ...        ...        ...  ..  ..  ...       ...   
Amateur 31        NaN  25.506401  12.596647  19.051524   4   0  ...  2.017564   
        32        NaN   2.773334  28.030267  15.401800   6   3  ...  2.006199   
        33        NaN   7.391318   7.616544   7.503931  24   4  ...  1.991175   
        34        NaN  -7.766953   5.115930  -1.325512   6   3  ...  2.107327   
        35        NaN -15.162277 -29.180718 -22.171497   6   1  ...  2.145059   

                 PFI       GFI       AFI      SvFI      ShFI       PAI  \
Premier 0   1.188291  1.296669  1.274910  1.089466  1.098730  0.916622   
        1   1.223315  1.158429  1.396269  1.150374  1.347118  1.031834   
        2   1.045441  1.360384  1.396810  0.681383  1.283500  1.033005   
        3   1.135048  1.265474  1.277084  1.265425  1.293373  1.033005   
        4   1.156718  1.369985  0.992061  1.158930  1.227201  0.954869   
...              ...       ...       ...       ...       ...       ...   
Amateur 31  0.784592  0.992131  0.573440  0.542605  0.619559  1.400098   
        32  0.526410  0.618231  0.758495  0.176298  0.498455  0.916463   
        33  0.602447  0.286816  0.705404  0.869146  0.481290  1.218560   
        34  0.508736  0.000000  0.000000  0.837605  0.316356  1.002316   
        35  0.393722  0.000000  0.000000  0.932154  0.310792  1.424034   

                 GAI       AAI      SvAI  
Premier 0   0.801327  0.714383  1.046274  
        1   0.806891  0.853599  1.271817  
        2   0.914766  0.889279  1.240775  
        3   0.914766  0.889279  1.240775  
        4   1.045979  0.889636  0.931626  
...              ...       ...       ...  
Amateur 31  2.238680  2.228702  0.549608  
        32  0.826646  1.036179  0.760854  
        33  1.517252  1.692567  0.910981  
        34  0.866895  0.781561  1.345377  
        35  2.421649  2.334175  0.649888  

[791 rows x 90 columns]

A1:CL792


C:\Users\MrDNA\AppData\Local\Temp\ipykernel_34432\3525909124.py:87: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  NewWorkBook.remove_sheet(WorkSheet)
